This notebook uses `scholarly` API (https://github.com/scholarly-python-package/scholarly) to enrich CS faculty info

In [1]:
from scrap_cs_faculty import *

## read CS faculty to be reprocessed

In [6]:
df = pd.read_csv("cs-faculty-no_scholar_id.csv")

In [7]:
df

,name,name_school
0,"Ada, Anil","Ada, Anil CMU"
1,"Berger, Daniel","Berger, Daniel CMU"
2,"Blum, Manuel","Blum, Manuel CMU"
3,"Brookes, Stephen","Brookes, Stephen CMU"
4,"Chrysanthis, Panos","Chrysanthis, Panos CMU"
...,...,...
224,Sanjit A. Seshia,Sanjit A. Seshia Berkeley
225,David A. Wagner,David A. Wagner Berkeley
226,Stella Yu,Stella Yu Berkeley
227,Gul A. Agha,Gul A. Agha UIUC


In [12]:
names = df.values

In [13]:
len(names), names 

(229,
 array([['Ada, Anil', 'Ada, Anil CMU'],
        ['Berger, Daniel', 'Berger, Daniel CMU'],
        ['Blum, Manuel', 'Blum, Manuel CMU'],
        ['Brookes, Stephen', 'Brookes, Stephen CMU'],
        ['Chrysanthis, Panos', 'Chrysanthis, Panos CMU'],
        ['DeYoung, Henry', 'DeYoung, Henry CMU'],
        ['Donahue, Chris', 'Donahue, Chris CMU'],
        ['Eckhardt, Dave', 'Eckhardt, Dave CMU'],
        ['Erdmann, Michael', 'Erdmann, Michael CMU'],
        ['Fatahalian, Kayvon', 'Fatahalian, Kayvon CMU'],
        ['Gligor, Virgil', 'Gligor, Virgil CMU'],
        ['Guruswami, Venkatesan', 'Guruswami, Venkatesan CMU'],
        ['Jahanian, Farnam', 'Jahanian, Farnam CMU'],
        ['Kaynar, Zeliha Dilsun', 'Kaynar, Zeliha Dilsun CMU'],
        ['Kohlbrenner, Anne', 'Kohlbrenner, Anne CMU'],
        ['Kosbie, David', 'Kosbie, David CMU'],
        ['Labrinidis, Alexandros', 'Labrinidis, Alexandros CMU'],
        ['Needham, Kit', 'Needham, Kit CMU'],
        ['Procaccia, Ariel', 'Procac

## fetch google scholar data for all CS faculty

In [22]:
ntest = -1  #  5 #  

In [23]:
pub_data = []
pub_error = []
for n, r in enumerate(names[:ntest]):
#     if n < 5: continue 
    name, author_org = r
    print(f"\n n, name = {n}, {name} ...")
    
    norm_auth_org = normalize_str(author_org)
    file_author = f"data/GScholar_{norm_auth_org}.json"

    try:
        search_query = scholarly.search_author(author_org)
        init_result = next(search_query)
    except Exception as ex:
        print(str(ex))
        init_result = None 
        
    if init_result is None:
        print(f"Failed to search_author: {author_org}")
        pub_error.append(r)
        continue

    scholar_id = init_result.get("scholar_id", "")
    if not scholar_id:
        print(f"Missing scholar_id")
        continue        
    
    # fetch data
    author = scholarly.fill(init_result)
    
    author_dict = {}
    # fill data cell
    author_dict["name"] = name 
    author_dict["file_author"] = file_author 
    author_dict["scholar_id"] = scholar_id 
    author_dict["affiliation"] = author.get("affiliation", "")
    author_dict["interests"] = "; ".join(author.get("interests", []))
    author_dict["url_author"] = get_scholar_page(scholar_id)
    author_dict["url_picture"] = author.get("url_picture", "")
    author_dict["url_homepage"] = author.get("homepage", "")
    author_dict["citedby"] = author.get("citedby", 0)
    author_dict["citedby5y"] = author.get("citedby5y", 0)
    author_dict["hindex"] = author.get("hindex", 0)
    author_dict["hindex5y"] = author.get("hindex5y", 0)
    author_dict["i10index"] = author.get("i10index", 0)
    author_dict["i10index5y"] = author.get("i10index5y", 0)
    author_dict["num_papers"] = len(author.get("publications", []))
    author_dict["num_coauthors"] = len(author.get("coauthors", []))
      
    # fill row 
    author_data = []
    for c in SCHOLAR_HEADER:
        author_data.append(author_dict.get(c))
        
    # accumulate row
    pub_data.append(author_data)
    
    # persist author data
    with open(Path(file_author), "w", encoding="utf-8") as f:
        f.write(json.dumps(author))
    
    delay = randint(1,5)
    sleep(delay)


 n, name = 0, Ada, Anil ...

Failed to search_author: Ada, Anil CMU

 n, name = 1, Berger, Daniel ...

Failed to search_author: Berger, Daniel CMU

 n, name = 2, Blum, Manuel ...

Failed to search_author: Blum, Manuel CMU

 n, name = 3, Brookes, Stephen ...

Failed to search_author: Brookes, Stephen CMU

 n, name = 4, Chrysanthis, Panos ...

Failed to search_author: Chrysanthis, Panos CMU

 n, name = 5, DeYoung, Henry ...

Failed to search_author: DeYoung, Henry CMU

 n, name = 6, Donahue, Chris ...

Failed to search_author: Donahue, Chris CMU

 n, name = 7, Eckhardt, Dave ...

Failed to search_author: Eckhardt, Dave CMU

 n, name = 8, Erdmann, Michael ...

Failed to search_author: Erdmann, Michael CMU

 n, name = 9, Fatahalian, Kayvon ...

Failed to search_author: Fatahalian, Kayvon CMU

 n, name = 10, Gligor, Virgil ...

Failed to search_author: Gligor, Virgil CMU

 n, name = 11, Guruswami, Venkatesan ...

Failed to search_author: Guruswami, Venkatesan CMU

 n, name = 12, Jahanian, 


Failed to search_author: Chris Hahn Stanford

 n, name = 104, Dolière Francis Somé ...

Failed to search_author: Dolière Francis Somé Stanford

 n, name = 105, Zhikun Zhang ...

 n, name = 106, Rediet Abebe ...

Failed to search_author: Rediet Abebe Berkeley

 n, name = 107, Babak Ayazifar ...

Failed to search_author: Babak Ayazifar Berkeley

 n, name = 108, Ruzena Bajcsy ...

Failed to search_author: Ruzena Bajcsy Berkeley

 n, name = 109, Brian A. Barsky ...

 n, name = 110, Lenore Blum ...

Failed to search_author: Lenore Blum Berkeley

 n, name = 111, Manuel Blum ...

Failed to search_author: Manuel Blum Berkeley

 n, name = 112, John F. Canny ...

Failed to search_author: John F. Canny Berkeley

 n, name = 113, Sarah Chasins ...

 n, name = 114, Alessandro Chiesa ...

Failed to search_author: Alessandro Chiesa Berkeley

 n, name = 115, Phillip Colella ...

Failed to search_author: Phillip Colella Berkeley

 n, name = 116, David E. Culler ...

Failed to search_author: David E. Cu


 n, name = 218, Michael Chang ...

Failed to search_author: Michael Chang Stanford

 n, name = 219, Serge Plotkin ...

Failed to search_author: Serge Plotkin Stanford

 n, name = 220, Shahar Dobzinski ...

Failed to search_author: Shahar Dobzinski Stanford

 n, name = 221, Marco Vassena ...

Failed to search_author: Marco Vassena Stanford

 n, name = 222, Michael J. Clancy ...

Failed to search_author: Michael J. Clancy Berkeley

 n, name = 223, David A. Patterson ...

Failed to search_author: David A. Patterson Berkeley

 n, name = 224, Sanjit A. Seshia ...

 n, name = 225, David A. Wagner ...

Failed to search_author: David A. Wagner Berkeley

 n, name = 226, Stella Yu ...

Failed to search_author: Stella Yu Berkeley

 n, name = 227, Gul A. Agha ...

Failed to search_author: Gul A. Agha UIUC


In [24]:
len(pub_data), len(pub_error)

(43, 185)

In [25]:
if len(pub_data) == len(pub_error):
    raise Exception("All searches failed!")

## write out xlsx

In [26]:
df_out = pd.DataFrame(pub_data, columns=SCHOLAR_HEADER)

In [27]:
df_out.head()

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,i10index5y,scholar_id,url_author,url_picture,url_homepage,file_author
0,"Zheng, Wenting",Carnegie Mellon University,System security,14,0,1870,10,10,1604,10,10,OeDgxpgAAAAJ,https://scholar.google.com/citations?user=OeDg...,https://scholar.google.com/citations?view_op=m...,http://wzheng.github.io/,data/GScholar_zheng_wenting_cmu.json
1,Carla Gomes,Universidade Federal de Pernambuco,inovação social; turismo de base comunitária; ...,154,0,1179,18,37,775,16,26,OULSUSEAAAAJ,https://scholar.google.com/citations?user=OULS...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_carla_gomes.json
2,Martha E. Pollack,"Provost, Professor of Computer Science and Eng...",Artificial Intelligence,218,0,15285,52,98,1979,18,34,5oU0wxcAAAAJ,https://scholar.google.com/citations?user=5oU0...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_martha_e_pollack.json
3,A. Kevin Tang,"University Professor, English Language and Lin...",computational linguistics; laboratory phonolog...,33,13,255,9,9,222,9,9,mGjFWCoAAAAJ,https://scholar.google.com/citations?user=mGjF...,https://scholar.google.com/citations?view_op=m...,http://www.kevintang.org/,data/GScholar_a_kevin_tang.json
4,Walker White,"History and Religious Studies Faculty, George ...",Byzantine Studies; Performing Arts; History; R...,7,0,78,3,3,47,3,1,lI13ZzgAAAAJ,https://scholar.google.com/citations?user=lI13...,https://scholar.google.com/citations?view_op=m...,,data/GScholar_walker_white.json


In [28]:
# import xlsxwriter
org = "extra"
file_xlsx = f"data/cs-faculty-gscholar-{org}.xlsx"
writer = pd.ExcelWriter(Path(file_xlsx), engine='xlsxwriter')
df_out.to_excel(writer, sheet_name=org, index=False)
writer.save()

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_16896\4238857146.py:6: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [29]:
file_xlsx

'data/cs-faculty-gscholar-extra.xlsx'